The Sentences_50Agree dataset from the Financial PhraseBank contains short sentences from financial news, this was implemented by Phd studens and industry experts. The idea is to show some sentences and anotate the feelings about that news to these expert. Finally, The result will be 100%, 80%, 70% of agreements. For the porpuse of this project 50% of agreement is using, because they can provide us a very complex semantic settings instead of 100% of agrement, for instance, that just give us obvius sentences.

In [1]:
from google.colab import files

uploaded = files.upload()

The snippet reads each line of Sentences_50Agree.txt, splits it at the last @ to extract the sentence and its label, and stores these pairs in a list of dictionaries.

In [3]:
import pandas as pd

# Path
ruta_txt = "/content/Sentences_75Agree.txt"

data = []
with open(ruta_txt, encoding="iso-8859-1") as f:
    for line in f:
        if "@" in line:
            sentence, label = line.rsplit("@", 1)
            data.append({"sentence": sentence.strip(), "label": label.strip()})

df = pd.DataFrame(data)
df.head()

,sentence,label
0,"According to Gran , the company has no plans t...",neutral
1,With the new production plant the company woul...,positive
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,"In the third quarter of 2010 , net sales incre...",positive
4,Operating profit rose to EUR 13.1 mn from EUR ...,positive


## a. BERT Model with Limited Data (0.5 points): Train a BERT-based model using only 32 labeled examples and assess its performance.

For this first task, we randomly select 32 labeled sentences from the Financial PhraseBank dataset, making sure the three sentiment classes (positive, neutral, negative) are balanced. The idea is to train a BERT model using only this small labeled set, just to see how it performs with very limited data.

In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=32, stratify=df["label"], random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

label_map = {"negative": 0, "neutral": 1, "positive": 2}
df_train["label"] = df_train["label"].map(label_map).astype(int)
df_test["label"] = df_test["label"].map(label_map).astype(int)

print(df_train)

                                             sentence  label
0   Cargotec 's business areas also include the co...      1
1   Operating profit was EUR 11.4 mn , up from EUR...      2
2         Sales by Seppala diminished by 6 per cent .      0
3   Simmons Elected DCUC Chairman PORTSMOUTH , N.H...      1
4   Operating profit rose from EUR 1.94 mn to EUR ...      2
5   The situation of coated magazine printing pape...      0
6   The new system , which will include 60 MC3090 ...      1
7   ( ADP News ) - Sep 30 , 2008 - Finnish securit...      2
8   At CapMan Haavisto will be responsible for Gro...      1
9   Pretax profit totalled EUR 80.8 mn , compared ...      2
10  `` We 've been feeling quite positive about th...      2
11  Sales are expected to increase in the end of t...      2
12  The passenger tunnel is expected to be put int...      1
13  The company pledged that the new software woul...      1
14  The government has instead proposed an exchang...      1
15  Alma Media 's operat

This block imports all the libraries that was needed and also set key-hyperparameters.

In [5]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer,
                          DataCollatorWithPadding, EarlyStoppingCallback)
import evaluate, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
import torch, random, os

data_collator = DataCollatorWithPadding(tokenizer=tok)
model_ckpt   = "bert-base-uncased"
num_labels   = 3
max_length   = 128
batch_size   = 64
seed         = 42

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

Hugging Face DatasetDict with two splits—train and test—by was created for train and split dataframes.

In [6]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 32
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3421
    })
})


The code tokenises every sentence: map() applies your tokenize function to each batch, trimming/padding to 128 tokens and discarding the raw text column.
It then renames the target column to labels, casts everything to PyTorch tensors, and print(dataset["train"][0]) shows one training example with input_ids, attention_mask, and its label ID.

In [7]:
tok = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
    return tok(batch["sentence"],
               truncation=True,
               padding="max_length",
               max_length=max_length)

dataset = (dataset
           .map(tokenize, batched=True, remove_columns=["sentence"])
           .rename_column("label", "labels"))
dataset.reset_format()
print(dataset["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/3421 [00:00<?, ? examples/s]

{'labels': 1, 'input_ids': [101, 6636, 26557, 1005, 1055, 2449, 2752, 2036, 2421, 1996, 11661, 8304, 7300, 2449, 2181, 10556, 19145, 2099, 1998, 1996, 3884, 6636, 8304, 1998, 12195, 7170, 8304, 7300, 2449, 2181, 6097, 17603, 20255, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

We loaded a BERT-base checkpoint and set it up for three-class sequence classification. Next, we went through every weight in the network and turned off gradient updates everywhere except the part called "classifier". That step froze the full encoder so only the small output layer remained trainable.

We did this because our project only had 32 labeled examples. Trying to update all 110 million BERT weights would have over-fit the data and slowed down training. By tuning just the lightweight classifier head (about one million parameters) we kept the pretrained language knowledge, used far less compute, and avoided letting the model simply memorize that tiny set.

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
            "bert-base-uncased",
            num_labels=3,
            problem_type="single_label_classification")

for n, p in model.named_parameters():
    if not n.startswith("classifier"):
        p.requires_grad = False

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We loaded pre-built scorers from evaluate—accuracy, F1, precision, and recall.
The compute_metrics function then took the modls logits, converted them to class IDs, and returned macro-averaged accuracy, F1, precision, and recall so that each class contributed equally despite any imbalance.

In [9]:
metric_acc   = evaluate.load("accuracy")
metric_f1    = evaluate.load("f1")
metric_prec  = evaluate.load("precision")
metric_rec   = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy"       : metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro"       : metric_f1 .compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision_macro": metric_prec.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall_macro"   : metric_rec .compute(predictions=preds, references=labels, average="macro")["recall"],
    }

First, we set up the training regime with TrainingArguments. We capped the run at 20 epochs but kept the mini-batch small (8) to prevent noisy gradient estimates from swamping the tiny 32-example training set. The learning rate (5 × 10⁻⁴) was deliberately high because only the lightweight classifier head remained trainable; a lower rate would have barely nudged its weights. We told the trainer to evaluate and save once per epoch so that every checkpoint corresponded to a full pass over the data, making comparisons of validation loss fair and consistent.

Next, we guarded against over-fitting by activating EarlyStoppingCallback with a patience of three epochs. With so few labelled examples the model could start memorising almost immediately; early stopping ensured we cut the run as soon as the validation loss stalled, while load_best_model_at_end=True automatically re-loaded the checkpoint that obtained the lowest loss. Matching evaluation_strategy and save_strategy to "epoch" was essential here: it guaranteed that the metric used for early stopping and the model selected as “best” were always drawn from the same evaluation snapshot.

Finally, we launched training through Trainer and then evaluated the preserved best checkpoint on the full test set. We reported macro-averaged accuracy, F1, precision, and recall so that each class—no matter how under-represented—contributed equally to the score. This workflow gave us a model tuned just enough to exploit BERT’s pretrained features without drifting into memorisation, and it produced performance metrics that reflected true generalisation rather than chance or class imbalance.

In [17]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
args = TrainingArguments(
    output_dir="./bert_fewshot",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    learning_rate=5e-4,
    eval_strategy="epoch",
    save_strategy="epoch",           # must match evaluation_strategy
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    seed=42,
)
early_stop = EarlyStoppingCallback(early_stopping_patience=3)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stop],
)


trainer.train()
metrics = trainer.evaluate()
print("\nFinal test metrics:")
for k, v in metrics.items():
    if k.startswith("eval_"):
        print(f"{k[5:]}: {v:.4f}")

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,1.042600,0.924638,0.618825,0.254845,0.206819,0.331922
2,0.961100,0.916648,0.619994,0.255143,0.206967,0.332549
3,0.906600,0.913149,0.620871,0.255365,0.207078,0.333020
4,0.931200,0.910313,0.621163,0.255439,0.207115,0.333177
5,0.917000,0.910291,0.621163,0.255439,0.207115,0.333177
6,0.889200,0.906857,0.621163,0.255439,0.207115,0.333177
7,0.924300,0.901643,0.621163,0.255439,0.207115,0.333177
8,0.867800,0.893287,0.621456,0.255513,0.207152,0.333333
9,0.876000,0.890312,0.621456,0.255513,0.207152,0.333333
10,0.870800,0.889816,0.621456,0.255513,0.207152,0.333333


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


Final test metrics:
loss: 0.8782
accuracy: 0.6215
f1_macro: 0.2555
precision_macro: 0.2072
recall_macro: 0.3333
runtime: 21.7846
samples_per_second: 157.0380
steps_per_second: 4.9120


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


After five full passes the early-stop kicked in: training loss kept drifting down but validation loss flattened at about 0.95, so the run cut itself short. The model ended up with ~0.59 accuracy on the big test split—almost identical to the share of the majority class—while macro F1 slid to ~0.29. That mismatch tells the story: with the encoder frozen and only 32 balanced shots, the classifier leaned hard on class 1 (the 60 % slice of the corpus) and largely ignored the smaller classes. It “looks good” if we only track accuracy, but macro precision and recall reveal the blind spot.

I expected that trade-off. Freezing the 110 M BERT weights kept the tiny dataset from being memorised, yet it also meant the model had limited freedom to carve decision boundaries for the minority labels. In short, it played safe—predict the label it has seen the most evidence for—and paid the macro-metric penalty. To push those macro numbers up I’d next unfreeze the last Transformer block, add class-weighted loss, or fabricate a few paraphrases for the scarce classes. Within the strict 32-example budget, though, this run gives me a clean baseline: decent overall accuracy, clear room for balanced-class improvement.

In [18]:
from transformers import BertTokenizer, BertForSequenceClassification

# Save the fine-tuned BERT model and tokenizer
model.save_pretrained("saved_teacher_model")
tok.save_pretrained("saved_teacher_model")

('saved_teacher_model/tokenizer_config.json',
 'saved_teacher_model/special_tokens_map.json',
 'saved_teacher_model/vocab.txt',
 'saved_teacher_model/added_tokens.json',
 'saved_teacher_model/tokenizer.json')